# Functional Class and Curvature Analysis
This notebook illustrates the complete analysis example using Here Map Content (HMC) attributes.

### Dependencies
* Catalogs: [rib-2](https://platform.here.com/data/hrn:here:data::olp-here:rib-2)
* Layers: [Topology Geometry](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/topology-geometry), [Road Attributes](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/road-attributes), [ADAS Attributes](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/adas-attributes)

### Workflow
* Start with a bounding box that contains portions of two tiles
* Extract geometry for the two tiles
* Keep only the segments that are (at least partially) in original bounding box
* Extract Functional Class (FC) attributes for those segments
* Select segments with FC = 3, 4, and 5
* Extract curvature for the survived segments
* Analyze average curvature
* Drop segments below the average curvature

## Get partition ID's from a given bounding box

In [1]:
from here.geotiles.heretile import in_bounding_box
from shapely.geometry import Point

zoom_level = 12
west = 13.41
east = 13.49
south = 52.50
north = 52.54
center = Point(west, south)
tileIds = list(in_bounding_box(west=west, south=south, east=east, north=north, level=zoom_level))
print("tileIds:", tileIds)

# partition ID's: string representation of tile IDs - this will be used for extracting attributes later
pids = [str(tile) for tile in tileIds]

tileIds: [23618402, 23618403]


## Visualize the bounding box and tile boundaries
- bounding box: Red
- tile boundaries: Gray

In [2]:
from here.inspector import inspect
from here.inspector.styles import Color
from shapely.geometry import Polygon

# construct a bounding box geojson
bb_polygon = Polygon([(west, south), (west, north), (east, north), (east, south), (west, south)])

# create inspect object with tile boundaries and the bounding box
inspect(layers={"bounding_box": [bb_polygon]},
        tiles=tileIds,
        center=center,
        zoom=11,
        layers_style={"bounding_box": Color.RED},
        tiles_style=Color.GRAY)

Map(center=[52.5, 13.41], controls=(LayersControl(options=['position']), ZoomControl(options=['position', 'zoo…

## Extract topology geometry and clip by bounding box

In [3]:
from here.platform import Platform
from here.content.hmc.hmc import HMC
from here.geopandas_adapter import GeoPandasAdapter
import pandas as pd
import geopandas as gpd

# create HMC object
hmc = HMC(Platform(adapter=GeoPandasAdapter()))

In [4]:
# get segments dataframe from given tiles
dfs = []
for tid in tileIds:
    dfs.append(hmc.topology_geometry.get_segments(tid))
segment_df = pd.concat(dfs)
segment_df

,partitionId,identifier,start_node_ref,end_node_ref,geometry,length
0,23618402,here:cm:segment:100005273,"partition_name: ""23618402""\nidentifier: ""here:...","partition_name: ""23618402""\nidentifier: ""here:...","LINESTRING (13.44399 52.51531, 13.44405 52.515...",53.97453
1,23618402,here:cm:segment:100005274,"partition_name: ""23618402""\nidentifier: ""here:...","partition_name: ""23618402""\nidentifier: ""here:...","LINESTRING (13.44347 52.51514, 13.44399 52.51531)",40.05030
2,23618402,here:cm:segment:100005285,"partition_name: ""23618402""\nidentifier: ""here:...","partition_name: ""23618402""\nidentifier: ""here:...","LINESTRING (13.40016 52.49682, 13.39965 52.496...",105.45831
3,23618402,here:cm:segment:100005286,"partition_name: ""23618402""\nidentifier: ""here:...","partition_name: ""23618402""\nidentifier: ""here:...","LINESTRING (13.40222 52.49641, 13.40076 52.49645)",99.25682
4,23618402,here:cm:segment:100005287,"partition_name: ""23618402""\nidentifier: ""here:...","partition_name: ""23618402""\nidentifier: ""here:...","LINESTRING (13.40076 52.49645, 13.40018 52.49646)",39.40682
...,...,...,...,...,...,...
8804,23618403,here:cm:segment:99973024,"partition_name: ""23618403""\nidentifier: ""here:...","partition_name: ""23618403""\nidentifier: ""here:...","LINESTRING (13.49611 52.54410, 13.49560 52.54424)",37.94505
8805,23618403,here:cm:segment:99973086,"partition_name: ""23618403""\nidentifier: ""here:...","partition_name: ""23618403""\nidentifier: ""here:...","LINESTRING (13.45018 52.47677, 13.45094 52.47736)",83.52769
8806,23618403,here:cm:segment:99983233,"partition_name: ""23618403""\nidentifier: ""here:...","partition_name: ""23618403""\nidentifier: ""here:...","LINESTRING (13.51952 52.49486, 13.51892 52.49584)",116.41634
8807,23618403,here:cm:segment:99990525,"partition_name: ""23618403""\nidentifier: ""here:...","partition_name: ""23618403""\nidentifier: ""here:...","LINESTRING (13.53283 52.48746, 13.53313 52.487...",457.78907


### Clip segments by the bounding box
Since the geometry objects for the segment dataframe is LineString, the intersection method will do the clipping. See the documentation of shapely: https://shapely.readthedocs.io/en/stable/manual.html

In [5]:
clipped_segments = segment_df.intersection(bb_polygon)
segment_in_bb = segment_df[clipped_segments.apply(lambda x: x.length != 0)]

In [6]:
# show the clipped segments and original tile boundaries
inspect(features=segment_in_bb["geometry"], tiles=tileIds, tiles_style=Color.GRAY)

Map(center=[52.5146484375, 13.447265625], controls=(LayersControl(options=['position']), ZoomControl(options=[…

## Get functional class attribute

In [7]:
# get functional_class attributes
fcs = []
for pid in pids:
    fcs.append(hmc.road_attributes.get_attribute(pid, "functional_class"))
fc_df = pd.concat(fcs)

# create a column called fc from attribute and drop attribute column
fc_df["fc"] = fc_df.apply(lambda x: x["attribute"].functional_class, axis=1)
fc_df = fc_df.drop(["attribute"], axis=1)

## Select FC = 3, 4, or 5

In [8]:
# select FC = 3, 4, 5
fc345_df = fc_df[fc_df["fc"] > 2]

In [9]:
# change partitionId type as string to be consistent with the one in attributes 
segment_in_bb = segment_in_bb.astype({"partitionId": str})
segment_in_bb.dtypes

partitionId         object
identifier          object
start_node_ref      object
end_node_ref        object
geometry          geometry
length             float64
dtype: object

In [10]:
# join two dataframe by partitionId, segmentId
segment_fc345_df = segment_in_bb.merge(fc345_df, how="inner", left_on=["partitionId", "identifier"],
                                       right_on=["partitionId", "segmentId"])
segment_fc345_df = segment_fc345_df.drop("identifier", axis=1)
segment_fc345_df.columns

Index(['partitionId', 'start_node_ref', 'end_node_ref', 'geometry', 'length',
       'segmentId', 'direction', 'start_offset', 'end_offset', 'fc'],
      dtype='object')

## Show the segments with FC = 3, 4, or 5

In [11]:
inspect(segment_fc345_df["geometry"])

Map(center=[52.519724999999994, 13.452165], controls=(LayersControl(options=['position']), ZoomControl(options…

## Show different colors for each FC

In [12]:
fcs = {}
for fc, gdf in segment_fc345_df.groupby("fc"):
    fcs[fc] = gdf["geometry"]
inspect(layers=fcs)

Map(center=[52.519724999999994, 13.452165], controls=(LayersControl(options=['position']), ZoomControl(options…

## Extract curvatures

In [13]:
# get segment curvatures (curvatures at the nodes are not used in this notebook example)
segment_curvatures = []
for pid in pids:
    segment_curvature, node_curvature = hmc.adas_attributes.get_attribute(pid, "curvature_heading")
    segment_curvatures.append(segment_curvature)
segment_curvature_df = pd.concat(segment_curvatures)

# create a column called curvature from attribute and drop attribute column
segment_curvature_df["curvature"] = segment_curvature_df.apply(lambda x: x["attribute"].curvature, axis=1)
segment_curvature_df = segment_curvature_df.drop("attribute", axis=1)
segment_curvature_df

,partitionId,segmentId,offset,curvature
0,23618402,here:cm:segment:146671023,0.111594,11014
1,23618402,here:cm:segment:146671023,0.246823,-202657
2,23618402,here:cm:segment:146671023,0.262919,-186693
3,23618402,here:cm:segment:146671023,0.505531,-580988
4,23618402,here:cm:segment:146671023,0.523570,-335825
...,...,...,...,...
12471,23618403,here:cm:segment:633360724,0.277245,194712
12472,23618403,here:cm:segment:633360724,0.683705,221743
12473,23618403,here:cm:segment:82556515,0.435010,207740
12474,23618403,here:cm:segment:185061412,0.409011,226037


## Select segments with high curvatures

In [14]:
# create a column called abs_curvature to calculate average curvature
segment_curvature_df["abs_curvature"] = segment_curvature_df.apply(lambda x: abs(x["curvature"]), axis=1)
average_curvature = segment_curvature_df["abs_curvature"].mean()

# select segments with curvature higher than average
highcurv_segments = segment_curvature_df[segment_curvature_df["abs_curvature"] > average_curvature]

## Join two dataframes
Inner join between the dataframe with FC=3,4,5 and the dataframe with high curvatures

In [15]:
# join two dataframe
segment_fc345_highcurv_df = segment_fc345_df.merge(highcurv_segments, on=["partitionId", "segmentId"])

In [16]:
# drop abs_curvature which was used to calculate average curvature
merged_df = segment_fc345_highcurv_df.drop("abs_curvature", axis=1)
merged_df.columns

Index(['partitionId', 'start_node_ref', 'end_node_ref', 'geometry', 'length',
       'segmentId', 'direction', 'start_offset', 'end_offset', 'fc', 'offset',
       'curvature'],
      dtype='object')

## Cleanup the merged dataframe
After merging two dataframe above, the result was quite duplicated. We want to have entries with start_offset <= offset <= end_offset

In [17]:
df = merged_df[((merged_df["start_offset"] <= merged_df["offset"]) &
               (merged_df["offset"] <= merged_df["end_offset"]))]

## Show the survived segments

In [18]:
inspect(df["geometry"])

Map(center=[52.52017, 13.451274999999999], controls=(LayersControl(options=['position']), ZoomControl(options=…

<span style="float:left; margin-top:3px;"><img src="https://www.here.com/themes/custom/here_base_theme_v2/logo.svg" alt="HERE Logo" height="60" width="60"></span><span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2021 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential information which may not be disclosed to others without the prior written consent of HERE.</sub></span>